#  **Pokemon_Type_Classification**(宝可梦第一属性的分类)
 
 >  **介绍**
    
*     宝可梦精灵通常具有属于它们属性的动物特征和显眼的颜色，例如火属性的通常是红色的、草属性通常是绿色的、暗属性通常是黑色的等等，因此颜色是表征它们属性的重要特征，但还是有例外。
    
*     同样的，动物特征如乌龟、鱼、翅膀等也表征了其属性，但也有例外，存在有乌龟特征的宝可梦其属性为火。

>  **本次实践进行的操作**
    
*     基于宝可梦的第一属性将宝可梦分类
*     利用CNN模型对宝可梦训练集及验证集进行训练，对测试集进行预测

**导入库**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
import seaborn as sns
import os
import tensorflow as tf
from tensorflow import keras as ks
from PIL import Image
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization, Lambda

**读取csv文件，预览数据**

In [ ]:
csv_data = pd.read_csv("../input/pokemon-images-and-types/pokemon.csv")
csv_data.info

**读取图像数据**

In [ ]:
images_data = "../input/pokemon-images-and-types/images/images/"

**将csv与图像对应**

In [ ]:
pokemon_filenames = os.listdir(images_data)

pnames, primary_types,secondary_types = [], [], []
for path in os.listdir(images_data):
        #割离宝可梦名字
        pokemon_name = path.split('.')[0]
        #割离宝可梦第一属性和第二属性
        row = csv_data[csv_data['Name']==pokemon_name]
        type1 = csv_data['Type1'][int(row.index.values)]
        type2 = csv_data['Type2'][int(row.index.values)]
        #增添到新列表
        pnames.append(pokemon_name.capitalize())
        primary_types.append(type1)  
        secondary_types.append(type2)
#创建新数据
pokemon_d = {'Image':pokemon_filenames, 'Pokemon':pnames, 'Primary_Type':primary_types, 'Secondary_Type':secondary_types}
pokemon_data = pd.DataFrame(pokemon_d)
#查看数据
pokemon_data
pokemon_data['Primary_Type'].value_counts()

**将测试集图像与csv文件对应**

In [ ]:
numbers = []
for i in range(1,csv_data.shape[0]+1):
    numbers.append(i)
csv_data['pkn'] = numbers#插入识别列
IMG_DIR = '/kaggle/input/pokemon-images-dataset/pokemon/pokemon'
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(IMG_DIR) if isfile(join(IMG_DIR, f))]#得到目录中所有文件(标题)

import re
dataframe_img = pd.DataFrame([])
images = []
pokemon_number = []
for img in onlyfiles:
    if not re.search('-', img):
        pkn = img.split('.')#将文件标题字符串分割为文件名和文件格式，如"['1','png']"
        n = re.sub("[^0-9]", "", pkn[0])#如果字符串中存在非数字，就置空
        path = str(img)
        images.append(path)
        pokemon_number.append(n)
dataframe_img['Image'] = images
dataframe_img['pkn'] = pokemon_number
dataframe_img['pkn'] = dataframe_img['pkn'].astype(int)#字符串转为数字
pokemon_test = csv_data.merge(dataframe_img, left_on='pkn', right_on='pkn')#合并文件
pokemon_test.rename(columns={'Name':'Pokemon','Type1':'Primary_Type','Type2':'Secondary_Type'},inplace=True)
pokemon_test['Primary_Type'].value_counts()

**分割数据集**

In [ ]:
#删除输出文件夹
import shutil
if os.path.exists('train/'):
    shutil.rmtree('train/') 
if os.path.exists('test/'):
    shutil.rmtree('test/') 
if os.path.exists('val/'):
    shutil.rmtree('val/') 

#创建训练集、测试集、验证集文件夹
from shutil import copyfile, copy2
if not os.path.exists('train/'):
    os.mkdir('train/')
if not os.path.exists('test/'):
    os.mkdir('test/')
if not os.path.exists('val/'):
    os.mkdir('val/')

#依据宝可梦第一属性创建子文件夹
for class_ in pokemon_data['Primary_Type'].unique():
    if not os.path.exists('train/'+str(class_)+'/'):
        os.mkdir('train/'+str(class_)+'/')
    if not os.path.exists('test/'+str(class_)+'/'):
        os.mkdir('test/'+str(class_)+'/')
    if not os.path.exists('val/'+str(class_)+'/'):
        os.mkdir('val/'+str(class_)+'/')

In [ ]:
#分割训练集和验证集，67%做训练集，33%做验证集 
X_train, X_val, y_train, y_val = train_test_split(
    pokemon_data, pokemon_data['Primary_Type'],test_size=0.33, stratify=pokemon_data['Primary_Type'], random_state=0)
#测试集
X_test, y_test = pokemon_test, pokemon_test['Primary_Type']

In [ ]:
#将分割出来的数据集分别依据第一属性存到文件夹中
for image,type_  in zip(images_data +'/' + X_train['Image'], y_train):
    copy2(image, 'train/'+type_)

for image,type_ in zip(IMG_DIR +'/' + X_test['Image'], y_test):
    copy2(image, 'test/'+type_)
    
for image,type_ in zip(images_data +'/' + X_val['Image'], y_val):
    copy2(image, 'val/'+type_)
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)


In [ ]:
datagen = ImageDataGenerator()#图片生成器初始化

train = datagen.flow_from_directory('train/')
test = datagen.flow_from_directory('test/')
val = datagen.flow_from_directory('val/')

**建立分类模型**
* CNN建模
* 使用3个卷积层、2D输入的最大池化层和丢弃法（Dropout）     

In [ ]:
#建模
def build():
    model = Sequential()#Sequential模型
    IMAGE_WIDTH = 256
    IMAGE_HEIGHT = 256
    IMAGE_CHANNELS = 3 #三通道
    #调用新建层
    model.add(Lambda(lambda x: x, input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))
    
    #以下设置的参数经过较多次测试，选的是结果最好的参数
    
    #卷积层(带有32个大小为3 * 3的过滤器的卷积层)，激活函数为relu，卷积层步长均为1，边缘填充padding为空
    #卷积后结果像素计算为：1+（输入像素大小-卷积核大小+2*padding）/步长。即为1+（256-3）/1=254
    model.add(Conv2D(32, (3, 3), activation='relu'))
    
    #批标准化减轻输入变化的影响，对神经元的输出进行归一化，激活函数的输入都在0附近，保证没有梯度的消失。
    #让每一层的值在有效的范围内传递下去，因为一层的输出是下一层的输入，从而使得每一层的输入分布对于不同批来说变化不大
    model.add(BatchNormalization())
    
    #池化核为3*3，步长为3，减小后期运算量，并且可以隐藏图像的绝对位置特征。
    #输出像素计算为：1+（输入像素大小-池化核大小）/步长。即为1+（254-3）/3=84
    model.add(MaxPooling2D(pool_size=(3, 3)))
    
    #使用Dropout，称为Dropout的正则化层，随机放弃30%的神经元，防止过拟合。
    model.add(Dropout(0.3))
    
    #第二次卷积
    #将32个图像转为64个图像
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.3))
    
    #第三次卷积
    #将64个图像转为128个图像
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.3))
    
    #默认按行的方向降维，把多维的输入一维化
    model.add(Flatten())
    #全连接层(根据特征的组合进行分类,大大减少特征位置对分类带来的影响)，共512个神经元
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    #加入50% Dropout
    model.add(Dropout(0.5))
    #输出层，18个神经元（对应18种第一属性）用于区别18个类
    #使用softmax激活函数进行转换，将神经元的输出转换为预测每个属性的概率
    model.add(Dense(18, activation='softmax')) 
    
    #查看模型摘要
    model.summary()
    
    #用compile方法设置训练模型
        #loss指定损失函数，本多分类任务指定交叉熵为损失函数
        #optimizer指定优化方法，选adam方法，使训练更快收敛，提高准确率
        #metrics模型观测参数（指标列表）。包含评估模型训练和测试性能的指标
    model.compile(loss='categorical_crossentropy', optimizer= "adam", metrics=['accuracy'])
    
    return model
model = build()
#开始训练
    #epochs表示执行25个训练周期
history = model.fit(train, epochs=25, validation_data=val)

**查看训练集和验证集的准确度及损失情况**

In [ ]:
plt.style.use('ggplot')

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.figure(figsize=(6, 5))

plt.plot(epochs, acc, 'r', label='training_accuracy')
plt.plot(epochs, val_acc, 'b', label='validation_accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('-----epochs--->')
plt.ylabel('Accuracy')
plt.legend()

plt.figure(figsize=(6, 5))

plt.plot(epochs, loss, 'r', label='training_loss')
plt.plot(epochs, val_loss, 'b', label='validation_loss')
plt.title('Training and Validation Loss')
plt.xlabel('----epochs--->')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
#预测测试集
predict = model.predict(test)

In [ ]:
#查看预测精度情况
from sklearn.metrics import classification_report
predict_frame = pd.DataFrame([])
predict_frame['category'] = np.argmax(predict, axis=-1)
labels = dict((v,k) for k,v in val.class_indices.items())
predict_frame['category'] = predict_frame['category'].replace(labels)
print(classification_report(y_test, predict_frame['category']))

> **结果分析1**
* 验证集准确度不高
* 从准确度和损失的线性图中可以看到存在过拟合的情况，其可能原因为部分第一属性类别的精灵较少，样本数量太少。
* 从测试集预测精度结果可以看到该模型对所有属性的分类精度都很低，即该模型不能很好的对所有属性的宝可梦进行分类
* 基于现有的数据尝试减少分类种数

**减少分类个数，尝试进行"Water"、"Fire"、"Electric"属性的分类**

In [ ]:
select = ['Water', 'Electric', 'Fire']
pokemon_test1 = pokemon_test[pokemon_test['Primary_Type'].isin(select)]
pokemon_data1 = pokemon_data[pokemon_data['Primary_Type'].isin(select)]
print(pokemon_data1['Primary_Type'].value_counts())
print(pokemon_test1['Primary_Type'].value_counts())

**创建新的训练集、测试集和验证集文件夹**

In [ ]:
#删除输出文件夹
if os.path.exists('strain/'):
    shutil.rmtree('strain/')
if os.path.exists('stest/'):
    shutil.rmtree('stest/')
if os.path.exists('sval/'):
    shutil.rmtree('sval/') 
    
#创建训练集、测试集、验证集文件夹
from shutil import copyfile, copy2
if not os.path.exists('strain/'):
    os.mkdir('strain/')
if not os.path.exists('stest/'):
    os.mkdir('stest/')
if not os.path.exists('sval/'):
    os.mkdir('sval/')
    
#依据宝可梦第一属性创建子文件夹
for class_ in pokemon_data1['Primary_Type'].unique():
    if not os.path.exists('strain/'+str(class_)+'/'):
        os.mkdir('strain/'+str(class_)+'/')
    if not os.path.exists('stest/'+str(class_)+'/'):
        os.mkdir('stest/'+str(class_)+'/')
    if not os.path.exists('sval/'+str(class_)+'/'):
        os.mkdir('sval/'+str(class_)+'/')

**分割数据集**

In [ ]:
#分割训练集和验证集 
X_strain, X_sval, y_strain, y_sval = train_test_split(
    pokemon_data1, pokemon_data1['Primary_Type'],test_size=0.33, stratify=pokemon_data1['Primary_Type'],random_state=0)

#测试集
X_stest, y_stest = pokemon_test1, pokemon_test1['Primary_Type']
X_strain

In [ ]:
#将分割出来的数据集分别依据第一属性存到文件夹中
for image,type_  in zip(images_data +'/' + X_strain['Image'], y_strain):
    copy2(image, 'strain/'+type_)

for image,type_ in zip(IMG_DIR +'/' + X_stest['Image'], y_stest):
    copy2(image, 'stest/'+type_)
    
for image,type_ in zip(images_data +'/' + X_sval['Image'], y_sval):
    copy2(image, 'sval/'+type_)
y_stest

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen1 = ImageDataGenerator()

strain = datagen1.flow_from_directory('strain/')
stest = datagen1.flow_from_directory('stest/')
sval = datagen1.flow_from_directory('sval/')


In [ ]:
#建模
def build1():
    model1 = Sequential()#Sequential模型
    IMAGE_WIDTH = 256
    IMAGE_HEIGHT = 256
    IMAGE_CHANNELS = 3 #RGB三通道
    #调用新建层
    model1.add(Lambda(lambda x: x, input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))
    
    model1.add(Conv2D(32, (3, 3), activation='relu'))
    model1.add(BatchNormalization())
    model1.add(MaxPooling2D(pool_size=(3, 3))) 
    model1.add(Dropout(0.3))
    
    model1.add(Conv2D(64, (3, 3), activation='relu'))
    model1.add(BatchNormalization())
    model1.add(MaxPooling2D(pool_size=(3, 3)))
    model1.add(Dropout(0.3))
    
    model1.add(Conv2D(128, (3, 3), activation='relu'))
    model1.add(BatchNormalization())
    model1.add(MaxPooling2D(pool_size=(3, 3)))
    model1.add(Dropout(0.3))
    
    model1.add(Flatten())
    model1.add(Dense(512, activation='relu'))
    model1.add(BatchNormalization())
    model1.add(Dropout(0.5))
    model1.add(Dense(3, activation='softmax'))#输出层，3个神经元（对应3种第一属性）用于区别3个类 
    
    #查看模型摘要
    model1.summary()
    
    model1.compile(loss='categorical_crossentropy', optimizer= "adam", metrics=['accuracy'])
    
    return model1
model1 = build1()
#开始训练
history1 = model1.fit(strain, epochs=25, validation_data=sval)

**查看准确度和损失**

In [ ]:
plt.style.use('ggplot')

acc = history1.history['accuracy']
val_acc = history1.history['val_accuracy']
loss = history1.history['loss']
val_loss = history1.history['val_loss']

epochs = range(len(acc))

plt.figure(figsize=(6, 5))

plt.plot(epochs, acc, 'r', label='training_accuracy')
plt.plot(epochs, val_acc, 'b', label='validation_accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('-----epochs--->')
plt.ylabel('Accuracy')
plt.legend()

plt.figure(figsize=(6, 5))

plt.plot(epochs, loss, 'r', label='training_loss')
plt.plot(epochs, val_loss, 'b', label='validation_loss')
plt.title('Training and Validation Loss')
plt.xlabel('----epochs--->')
plt.ylabel('Loss')
plt.legend()

plt.show()

从结果可以看到改为分3类后，分类准确度升高了，从0.2左右升到了0.6-0.7左右

In [ ]:
#对测试集进行预测
predict1 = model1.predict(stest)

In [ ]:
#查看各种类的精度
from sklearn.metrics import classification_report
predict_frame1 = pd.DataFrame([])
predict_frame1['category'] = np.argmax(predict1, axis=-1)
labels = dict((v,k) for k,v in sval.class_indices.items())
predict_frame1['category'] = predict_frame1['category'].replace(labels)
print(classification_report(y_stest, predict_frame1['category']))

可以看到该分类模型对水属性的宝可梦分类精确率和召回率都较高

**查看部分分类失败的宝可梦**

In [ ]:
def show_wrong_classification(y_stest, predict1, pokemon_test1):
    tmp = pokemon_test1[pokemon_test1.index.isin(y_stest.index)]
    fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(20, 20))
    i=0
    for imag, true, pred in zip(IMG_DIR +'/' + tmp['Image'], tmp['Primary_Type'], predict1):
        if true!=pred:
            if i < 4:
                img = Image.open(imag)
                fig = plt.figure()
                ax[i].imshow(img)
                ax[i].set_title('Actual:'+str(true)+'\n'+'predict:'+ str(pred))
                i+=1

In [ ]:
show_wrong_classification(y_stest, predict_frame1['category'], pokemon_test1)

> **结果分析2**
* 从结果可以看到模型将部分宝可梦属性识别错误，识别存在一定问题。
* 不同属性的宝可梦识别分类效果差，但从视觉上忽略它们的动物特征，可发现部分宝可梦的颜色是相似的。


**查看像素rgb通道在火、水和电属性的宝可梦中是如何分布的**

In [ ]:
#查看像素在3个颜色通道的分布
import seaborn as sns
import matplotlib.pyplot as plt

fig, axes = plt.subplots(3,3,figsize=(15,10))
k =0
import cv2
list_b =[]
list_r = []
list_g = []
from tqdm import tqdm
for type_ in tqdm(pokemon_test1['Primary_Type'].unique()):
    tmp = pokemon_test1[pokemon_test1['Primary_Type']==type_]
    for img in IMG_DIR +'/' + tmp['Image']:
        img = cv2.imread(img)
        b, g, r = cv2.split(img)
        color = 'blue'
        for i in b:
            for j in i:
                if j != 0:
                    list_b.append(j)
        color = 'green'
        for i in g:
            for j in i:
                if j != 0:
                    list_g.append(j)
        color = 'red'
        for i in r:
            for j in i:
                if j != 0:
                    list_r.append(j)
    sns.distplot(list_g, ax=axes[k, 0], color='g')
    sns.distplot(list_b, ax=axes[k, 1], color='b')
    sns.distplot(list_r, ax=axes[k, 2], color='r')
    axes[k, 0].set_title('Pokemon type color channel ' + type_)
    if type_ =='Fire':
        list_g_f = list_g
        list_b_f = list_b
        list_r_f = list_r
    elif type_=='Water':
        list_g_w = list_g
        list_b_w = list_b
        list_r_w = list_r
    else:
        list_g_e = list_g
        list_b_e = list_b
        list_r_e = list_r
    list_b =[]
    list_r = []
    list_g = []
    
    k += 1

> **结果分析3**
*     三种属性的绿色通道和红色通道内像素分布相似。蓝色通道是分布差异最大的通道，Fire属性的宝可梦在200 ~ 255像素值之间出现率较低，Water属性的相反，Electric属性的宝可梦像素值分布较为均匀。
*     水属性、雷属性和火属性的部分宝可梦都有较高红色的表征，因此会导致错误的分类识别。

**对一张图片进行识别**

In [ ]:
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
#加载图片
num=171

#上方测试集识别分类有误的图片标号：3、4、5、6、7、8  
#火属性识别（部分）：156、320  ， 321、322
#水属性识别（部分）：157、117、224、319、365、367、368  ， 79、115、118、119、221、340
#雷属性识别（部分）：识别正确编号：99、100、171、310、458、513   ， 24、134 
#有红色表征的其他属性识别为火属性的部分图片编号：128、222、309、339
#由于模型稳定性较弱，上述分类编号可能会有误

img = Image.open(IMG_DIR +'/' + X_test['Image'][num])#打开输出图片
img1 = load_img(IMG_DIR +'/' + X_test['Image'][num])#加载识别图片
data = img_to_array(img1)
samples = expand_dims(data, 0)

#调用三分类模型进行预测
predict1 = model1.predict(samples)
predict_frame1 = pd.DataFrame([])
predict_frame1['category'] = np.argmax(predict1, axis=-1)
labels = dict((v,k) for k,v in sval.class_indices.items())
#预测结果和真实结果
predict_frame1['category'] = predict_frame1['category'].replace(labels)
pred=predict_frame1['category'][0]
true=X_test['Primary_Type'][num]
#展示
fig = plt.figure()
plt.imshow(img)
plt.title('Actual: ' + str(true) + '\n' + 'Predict: ' + str(pred))

> **结果分析4**
* 通过单张图片的识别结果，可以发现部分有红色表征的水、电属性的宝可梦识别有误。同时，部分有红色表征的宝可梦识别正确（三种属性均有识别正确的）。
* 模型稳定度不高，每训练一次模型，预测的结果都有或多或少的差别。
* 颜色对识别结果存在一定的影响，但又不可或缺。

**尝试将图像灰度化**

In [ ]:
#将第一属性转为由数字表示
TYPES = {
    0:"Water",
    1:"Electric",
    2:"Fire"}
key_list = list(TYPES.keys())
val_list = list(TYPES.values())
def replace_type(x):
    position = val_list.index(x)
    return key_list[position]

y_Strain = y_strain
y_Sval = y_sval
y_Stest = y_stest
y_Strain = y_Strain.apply(lambda x: replace_type(x))
y_Stest= y_Stest.apply(lambda x: replace_type(x))
y_Sval = y_Sval.apply(lambda x: replace_type(x))


In [ ]:
from numpy import expand_dims
#明度法图像灰度化
def imgGray_Lum(im1):
    imgarray1 = np.array(im1, dtype=np.float32)#以浮点型读取图像数据
    rows = im1.shape[0]
    cols = im1.shape[1]
    for i in range(rows):
        for j in range(cols):
            imgarray1[i, j, :] = (imgarray1[i, j, 0] * 0.299 + imgarray1[i, j, 1] * 0.587 + imgarray1[i, j, 2] * 0.114)
    return imgarray1.astype(np.uint8)#保存为uint8类型图像

bs = list()
bs1 = list()
bs2 = list()

#训练集灰度化
for imag,r in zip(X_strain['Image'],X_strain['Image'].index.values):
    I = Image.open(images_data +'/' + X_strain['Image'][r])
    I = I.resize((256,256),Image.ANTIALIAS) #改变图像大小
    I = I.convert('RGB')
    I = np.array(I)
    I = imgGray_Lum(I)
    #将n个三维数组拼成一个四维数组，此处效果为得到数组shape为（n,256,256,3）
    bs.append(I[np.newaxis, :])
    data = np.concatenate(bs, axis=0)
#将训练集转换为数组表示
X_Strain = np.empty((data.shape[0],256,256,3))
for i in range(data.shape[0]):
    X_Strain[i] = np.array(data[i])
y_Strain = np.array(y_Strain)

#验证集灰度化
for imag,r in zip(X_sval['Image'],X_sval['Image'].index.values):
    Iv = Image.open(images_data +'/' + X_sval['Image'][r])
    Iv = Iv.resize((256,256),Image.ANTIALIAS) #改变图像大小
    Iv = Iv.convert('RGB')
    Iv = np.array(Iv)
    Iv = imgGray_Lum(Iv)
    bs1.append(Iv[np.newaxis, :])
    data1 = np.concatenate(bs1, axis=0)
#将验证集转换为数组表示    
X_Sval = np.empty((data1.shape[0],256,256, 3))
for i in range(data1.shape[0]):
    X_Sval[i] = np.array(data1[i])   
y_Sval = np.array(y_Sval)

#测试集灰度化
for imag,r in zip(X_stest['Image'],X_stest['Image'].index.values):
    Ie = Image.open(IMG_DIR+'/' + X_stest['Image'][r])
    Ie = Ie.convert('RGB')
    Ie = np.array(Ie)
    Ie = imgGray_Lum(Ie)
    bs2.append(Ie[np.newaxis, :])
    data2 = np.concatenate(bs2, axis=0)
#将测试集转换为数组表示   
X_Stest = np.empty((data2.shape[0],256,256,3))
for i in range(data2.shape[0]):
    X_Stest[i] = np.array(data2[i])   
y_Stest = np.array(y_Stest)
print(y_Stest)

**建立灰度图的CNN模型**

In [ ]:
#建模
def build2():
    model2 = Sequential()#Sequential模型
    IMAGE_WIDTH = 256
    IMAGE_HEIGHT = 256
    IMAGE_CHANNELS = 3 #三通道
    
    #调用新建层
    model2.add(Lambda(lambda x: x, input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))
    
    model2.add(Conv2D(32, (3, 3), activation='relu'))
    model2.add(BatchNormalization())
    model2.add(MaxPooling2D(pool_size=(3, 3))) 
    model2.add(Dropout(0.3))
    
    model2.add(Conv2D(64, (3, 3), activation='relu'))
    model2.add(BatchNormalization())
    model2.add(MaxPooling2D(pool_size=(3, 3)))
    model2.add(Dropout(0.3))
    
    model2.add(Conv2D(128, (3, 3), activation='relu'))
    model2.add(BatchNormalization())
    model2.add(MaxPooling2D(pool_size=(3, 3)))
    model2.add(Dropout(0.3))
    
    model2.add(Flatten())
    model2.add(Dense(512, activation='relu'))
    model2.add(BatchNormalization())
    model2.add(Dropout(0.5))
    model2.add(Dense(3, activation='softmax'))#输出层，3个神经元（对应3种第一属性）用于区别3个类 
    
    #查看模型摘要
    model2.summary()
    
    #稀疏类别交叉熵损失，自动将其中一方不是one-hot表示的数据转换为one-hot表示
    model2.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer= "adam", metrics=['accuracy'])
    
    return model2


**训练模型**

In [ ]:
model2 = build2()
history2 =  model2.fit(X_Strain,y_Strain, epochs=25, validation_data=(X_Sval,y_Sval))#训练模型

In [ ]:
#查看准确度和损失
plt.style.use('ggplot')
acc = history2.history['accuracy']
val_acc = history2.history['val_accuracy']
loss = history2.history['loss']
val_loss = history2.history['val_loss']

epochs = range(len(acc))

plt.figure(figsize=(6, 5))

plt.plot(epochs, acc, 'r', label='training_accuracy')
plt.plot(epochs, val_acc, 'b', label='validation_accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('-----epochs--->')
plt.ylabel('Accuracy')
plt.legend()

plt.figure(figsize=(6, 5))

plt.plot(epochs, loss, 'r', label='training_loss')
plt.plot(epochs, val_loss, 'b', label='validation_loss')
plt.title('Training and Validation Loss')
plt.xlabel('----epochs--->')
plt.ylabel('Loss')
plt.legend()

plt.show()

**对灰度测试集进行预测**

In [ ]:
#对灰度测试集进行预测
predict3 = model2.predict(X_Stest)

#查看各种类的精度
from sklearn.metrics import classification_report
predict_frame3 = pd.DataFrame([])
predict_frame3['category'] = np.argmax(predict3, axis=-1)
labels2 = dict((v,k) for k,v in sval.class_indices.items())
predict_frame3['category'] = predict_frame3['category'].replace(labels2)
print(classification_report(y_stest, predict_frame3['category']))

> **结果分析5**
* 将图片灰度化后进行训练，验证集的的准确度与未进行灰度化时相比降低了。
* 测试集的精确度与原先相比略有波动。
* 将图片灰度化后训练效果一般，甚至会变差。

**查看单张灰度图片的识别情况**

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array#加载图片
num = 6

im = X_Stest[num]#打开输出图片
samples1 = expand_dims(im, 0)

#调用三分类灰度模型进行预测
predict2 = model2.predict(samples1)
predict_frame2 = pd.DataFrame([])
predict_frame2['category'] = np.argmax(predict2, axis=-1)
labels1 = dict((v,k) for k,v in sval.class_indices.items())
#预测结果和真实结果
predict_frame2['category'] = predict_frame2['category'].replace(labels1)
pred1=predict_frame2['category'][0]
true1=TYPES[y_Stest[num]]
#展示
fig = plt.figure()
plt.imshow(im.astype("uint8"))
plt.title('Actual: ' + str(true1) + '\n' + 'Predict: ' + str(pred1))


> **结果分析6**
* 将图片灰度化后进行预测，结果发现，原先有红色特征的被识别为火属性的水属性宝可梦大部分都识别准确了，即减少RGB的影响能使模型更关注于动物特征，但仍有部分被识别为电属性。
* 原先识别正确和识别错误的宝可梦，现在有正确->错误、错误->正确的情况出现，也有部分仍然识别正确/错误。
* 从上述结果可以看到，减少RGB的影响对有明显红色特征的宝可梦识别有较大帮助，能够较正确的识别水和电属性，但同时对火属性的宝可梦识别较差。对于部分水和电属性的宝可梦，减少RGB影响对其识别影响较小，即它们的其他属性仍较为相似。
* 该模型稳定性不高，重新运行模型预测结果会与之前的结果不同。
* 颜色和动物特征对宝可梦的属性分类都有一定的影响，在识别时会有一定困难。

# **总结**

* 本次实践所建立的CNN模型稳定性较弱，对宝可梦属性的识别效果一般，特别是对较多属性的识别，其原因可能为数据样本较少及图片本身较难识别。
* 对于宝可梦的属性识别，其颜色特征和动物特征都十分重要，从灰度化图像后的训练结果看，颜色特征所占的比重是较大的。
* 在本次实践中，尝试对图像进行增强（旋转、放大、调亮）但效果不明显，红色特征会干扰识别分类。
* 从本次实践中可以发现，对于需要颜色特征参与识别分类的数据，其数据如何处理、训练模型如何改进是需要考虑的，可能需要更多的数据集或者更深的网络。
* 由于时间有限，本次实践未进行更深度的探究，部分结果及分析可能不全面。